# Bricklayer Robot

In [1]:
# Bookkeeping
# Python path for the local Drake and Manipulation
import sys

# Cool stuff
import shutil
import os as os
import matplotlib.pyplot as plt
import mpld3
import numpy as np
from IPython.display import HTML, display
from manipulation import running_as_notebook, FindResource
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import MakeManipulationStation
from pydrake.all import (AddMultibodyPlantSceneGraph, AngleAxis, BasicVector,
                         ConstantVectorSource, DiagramBuilder,
                         FindResourceOrThrow, Integrator, JacobianWrtVariable,
                         LeafSystem, MeshcatVisualizer,
                         MeshcatVisualizerParams, MultibodyPlant,
                         MultibodyPositionToGeometryPose, Parser,
                         PiecewisePose, Quaternion, RigidTransform,
                         RollPitchYaw, RotationMatrix, SceneGraph, Simulator,
                         StartMeshcat, TrajectorySource, GenerateHtml, GetDrakePath, PiecewisePolynomial)

In [2]:
# Some paths
kProjectDir = os.getcwd()
print("Working in directory ", kProjectDir)

Working in directory  /home/nikita/MIT_6_4212/project


In [3]:
# Hack: move our model packages to the manipulation/ folder to allow Drake to find them
shutil.copyfile(kProjectDir + "/models/brick.dmd.yaml", kProjectDir + "/../manipulation/manipulation/models/brick.dmd.yaml")
shutil.copyfile(kProjectDir + "/models/real_brick.sdf", kProjectDir + "/../manipulation/manipulation/models/real_brick.sdf")
shutil.copyfile(kProjectDir + "/models/ground_model.sdf", kProjectDir + "/../manipulation/manipulation/models/ground_model.sdf")

'/home/nikita/MIT_6_4212/project/../manipulation/manipulation/models/ground_model.sdf'

In [4]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [5]:
# System configs and playground
# Playground: robot and basement
model_directives = """
directives:
- add_directives:
    file: package://manipulation/iiwa_and_wsg.dmd.yaml

- add_frame:
    name: ground_origin
    X_PF:
      base_frame: world
      rotation: !Rpy { deg: [0.0, 0.0, 0.0 ]}
      translation: [0.0, 0.0, 0.0]

- add_model:
    name: ground
    file: package://manipulation/ground_model.sdf

- add_weld:
    parent: ground_origin
    child: ground::ground_base

"""
# Brick template
brick_string = """
- add_model:
    name: brick$
    file: package://manipulation/real_brick.sdf

"""

# Generate many bricks under the scene
kNumberOfBricks = 100 # @param
for i in range(0, kNumberOfBricks):
    brick_string_new = brick_string.replace("brick$", "brick_" + str(i))
    model_directives = model_directives + brick_string_new

# Brick warehouse config
kWhLocation = np.array([2.5, 2.5, 0.15]) # location of the warehouse (corner), # @param
kWhSize = np.array([5, 5, 5]) # size of the warehouse grid in cells, # @param
if (kWhSize.prod() < kNumberOfBricks):
    assert False, "The warehouse is too small to fit all bricks"

# Brick source
X_WBrickSource = RigidTransform(np.array([0.5, 0.5, 0.05])) # @param

In [41]:
#
# iiwa robot class
#
class IIWA_Painter():
    def __init__(self, traj_in=None):
        builder = DiagramBuilder()

        # set up the system of manipulation station
        self.station = MakeManipulationStation(model_directives=model_directives)
        builder.AddSystem(self.station)

        # set up plant
        self.plant = self.station.GetSubsystemByName("plant")

        # optionally add trajectory source
        if traj_in is not None:
            # traj and PseudoInverseController
            traj = traj_in[0]
            traj_V_G = traj.MakeDerivative()
            V_G_source = builder.AddSystem(TrajectorySource(traj_V_G))
            self.controller = builder.AddSystem(
                PseudoInverseController(self.plant))
            builder.Connect(V_G_source.get_output_port(),
                            self.controller.GetInputPort("V_G"))

            # integrator and controller
            self.integrator = builder.AddSystem(Integrator(7))
            builder.Connect(self.controller.get_output_port(),
                            self.integrator.get_input_port())
            builder.Connect(self.integrator.get_output_port(),
                            self.station.GetInputPort("iiwa_position"))
            builder.Connect(
                self.station.GetOutputPort("iiwa_position_measured"),
                self.controller.GetInputPort("iiwa_position"))

            # and trajectory source for the grip fingers as well
            finger_traj = traj_in[1]
            wsg_source = builder.AddSystem(TrajectorySource(finger_traj))
            wsg_source.set_name("wsg_command")
            builder.Connect(wsg_source.get_output_port(), self.station.GetInputPort("wsg_position"))

        # visualization
        params = MeshcatVisualizerParams()
        params.delete_on_initialization_event = False
        self.visualizer = MeshcatVisualizer.AddToBuilder(
            builder, self.station.GetOutputPort("query_object"), meshcat, params)

        # build and add diagram
        self.diagram = builder.Build()    
        self.gripper_frame = self.plant.GetFrameByName('body')
        self.world_frame = self.plant.world_frame()

        # resolve context
        context = self.CreateDefaultContext()
        self.diagram.Publish(context)

    # Helper to visualize frame
    def visualize_frame(self, name, X_WF, length=0.15, radius=0.006):
        AddMeshcatTriad(meshcat, "painter/" + name,
                        length=length, radius=radius, X_PT=X_WF)

    # Helper to create default context
    def CreateDefaultContext(self):
        context = self.diagram.CreateDefaultContext()
        plant_context = self.diagram.GetMutableSubsystemContext(
            self.plant, context)
        station_context = self.diagram.GetMutableSubsystemContext(
            self.station, context)

        # provide initial states
        q0 = np.array([ 1.40666193e-05,  1.56461165e-01, -3.82761069e-05,
                       -1.32296976e+00, -6.29097287e-06,  1.61181157e+00, -2.66900985e-05])
        iiwa = self.plant.GetModelInstanceByName("iiwa")
        self.plant.SetPositions(plant_context, iiwa, q0)
        self.plant.SetVelocities(plant_context, iiwa, np.zeros(7))
        wsg = self.plant.GetModelInstanceByName("wsg")
        self.plant.SetPositions(plant_context, wsg, [-0.05, 0.05])
        self.plant.SetVelocities(plant_context, wsg, [0, 0])        

        if hasattr(self, 'integrator'):
            self.integrator.set_integral_value(
                self.integrator.GetMyMutableContextFromRoot(context), q0)

        return context
    
    def reset_integrator(self, context, q0):
        if hasattr(self, 'integrator'):
            self.integrator.set_integral_value(
                self.integrator.GetMyMutableContextFromRoot(context), q0)
            
    def get_q0(self, contex):
        station_context = self.station.GetMyContextFromRoot(context)
        return self.station.GetOutputPort("iiwa_position_measured").Eval(station_context)
    
    # Helper to get current grip position
    def get_X_WG(self, context=None):

        if not context:
            context = self.CreateDefaultContext()
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        X_WG = self.plant.CalcRelativeTransform(
                    plant_context,
                    frame_A=self.world_frame,
                    frame_B=self.gripper_frame)
        return X_WG
    
    # Lock bricks
    def lock_brick(self, context, brick_num):
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        brick_body = self.plant.GetBodyByName("base_link", self.plant.GetModelInstanceByName("brick_" + str(brick_num)))
        brick_body.Lock(plant_context)
        
    # Unlock bricks
    def unlock_brick(self, context, brick_num):
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        brick_body = self.plant.GetBodyByName("base_link", self.plant.GetModelInstanceByName("brick_" + str(brick_num)))
        brick_body.Unlock(plant_context)
    
    # Generate new brick
    def move_brick(self, context, X_WBrickSource, brick_num):
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        brick_body = self.plant.GetBodyByName("base_link", self.plant.GetModelInstanceByName("brick_" + str(brick_num)))
        self.plant.SetFreeBodyPose(plant_context, brick_body, X_WBrickSource)
        self.visualize_frame("brick_source", X_WBrickSource)

    # Run simulation
    def paint(self, simulator, context, sim_duration=20.0):
        if context == None:
            context = self.CreateDefaultContext()
        
        if simulator == None:
            simulator = Simulator(self.diagram, context)
            simulator.set_target_realtime_rate(1.0)

        duration = sim_duration if running_as_notebook else 0.01
        simulator.AdvanceTo(duration)
        
        return (simulator, context)

#
# PseudoInverseController class
#
class PseudoInverseController(LeafSystem):
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.V_G_port = self.DeclareVectorInputPort("V_G", BasicVector(6))
        self.q_port = self.DeclareVectorInputPort("iiwa_position", BasicVector(7))
        self.DeclareVectorOutputPort("iiwa_velocity", BasicVector(7),
                                     self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()

    def CalcOutput(self, context, output):
        V_G = self.V_G_port.Eval(context)
        q = self.q_port.Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, JacobianWrtVariable.kV,
            self._G, [0,0,0], self._W, self._W)
        J_G = J_G[:,self.iiwa_start:self.iiwa_end+1] # Only iiwa terms.
        v = np.linalg.pinv(J_G).dot(V_G) #important
        output.SetFromVector(v)

#
# Trajectory class: define and manipulate trajectories
#
class Trajectory():
    def __init__(self):
        # [timestamp, pose, grip_pose, metainfo, breakpoint]
        self.traj = []
        # [timestamp]
        self.breakpoints = []

    def get_traj(self):
        return self.traj
    
    def get_breakpoints(self):
        return self.breakpoints

    # Add intermediate point in the trajectory
    def append_point(self, timestamp, pose, grip_pose, metainfo, breakpoint = False, calibration_q0 = None, with_merge = True):
        base_t = self.traj[-1][0] if len(self.traj) else 0
        ts = base_t + timestamp if with_merge else timestamp
        self.traj.append([ts, pose, grip_pose, metainfo, breakpoint])
        if breakpoint:
            self.breakpoints.append(ts)

    # Merge this trajectory with `traj_to_merge`, putting traj after
    # Add a small constant as a gap between trajectories
    def merge_in(self, traj_to_merge):
        # Merge trajectories
        gap = 0.0001
        base_t = self.traj[-1][0] + gap
        for trj in traj_to_merge.get_traj():
            self.append_point(base_t + trj[0], trj[1], trj[2], trj[3], trj[4]);

    # Shrink trajectories by scaling all timestamps for each point
    def slow_down(self, k):
        for trj in self.traj:
            trj[0] = trj[0] * k
        self.breakpoints = [x * k for x in self.breakpoints]

    # Dump each point based on mask corresponding to
    # [timestamp, pose, grip_pose, metainfo, breakpoint]
    def dump_trajectories(self, mask=[False, False, False, False, False]):
        for trj in self.traj:
            dump = list(filter(lambda x: x[1] == True, zip(trj, mask)))
            print([x[0] for x in dump])

    def dump_breakpoints(self):
        for bp in self.breakpoints:
            print(bp)
            
    # Form iiwa grip and finger trajectories
    def form_iiwa_traj(self):
        #
        traj = PiecewisePose.MakeLinear(np.array([t[0] for t in self.traj]), 
                                        np.array([t[1] for t in self.traj]))
        #
        finger_traj = PiecewisePolynomial.FirstOrderHold([self.traj[0][0], self.traj[1][0]], 
                                                         np.hstack([[self.traj[0][2]], [self.traj[1][2]]]))
        for i in range(2, len(self.traj)):
            finger_traj.AppendFirstOrderSegment(self.traj[i][0], self.traj[i][2])
        #
        return (traj, finger_traj)
    
#
# Trajectory builder: make trajectories described by `class Trajectory`
#
class TrajectoryBuilder:
    # Init trajectory builder with the brick source at X_WBrickSource
    def __init__(self, system, X_WBrickSource):
        #
        self.X_WBrickSource = X_WBrickSource
        self.system = system

        # define some constant reference poses
        self.X_BrickSourcePreG = RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2),
                                               np.array([0, 0, 0.35]))
        self.X_BrickSourceG = RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2),
                                            np.array([0, 0, 0.12]))
        self.X_BrickTargetPreG = RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2),# @ RotationMatrix.MakeYRotation(-np.pi/2),
                                               np.array([0, 0, 0.35]))
        self.X_BrickTargetG = RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2),# @ RotationMatrix.MakeYRotation(-np.pi/2),
                                            np.array([0, 0, 0.12]))
        self.finger_opened = np.array([0.15]);
        self.finger_closed = np.array([0.05]);

        #
        self.trajectory = Trajectory()
        
    def gen_initial_traj(self):
        self.trajectory.append_point(0,
                                     self.system.get_X_WG(),
                                     self.finger_opened,
                                     'initial')
        self.trajectory.append_point(2,
                                     self.X_WBrickSource @ self.X_BrickSourcePreG,
                                     self.finger_opened,
                                     'calibration',
                                     True)
        
    def gen_grab_brick_traj(self):
        self.trajectory.append_point(0.0001,
                                     self.X_WBrickSource @ self.X_BrickSourcePreG,
                                     self.finger_opened,
                                     'pre-grab')
        self.trajectory.append_point(2,
                                     self.X_WBrickSource @ self.X_BrickSourceG,
                                     self.finger_opened,
                                     'grab')
        self.trajectory.append_point(2.5, self.X_WBrickSource @ self.X_BrickSourceG,
                                     self.finger_closed,
                                     'grab and close')

    # Generate move trajectory to the point X_WBrickTarget
    def gen_move_to_place_traj(self, X_WBrickTarget):
        self.trajectory.append_point(0.0001,
                                     self.X_WBrickSource @ self.X_BrickSourceG,
                                     self.finger_closed,
                                     'carry_begin')
        self.trajectory.append_point(4,
                                     X_WBrickTarget @ self.X_BrickTargetPreG,
                                     self.finger_closed,
                                     'carry_end')

    def gen_place_brick_traj(self, X_WBrickTarget):
        self.trajectory.append_point(0.0001,
                                     X_WBrickTarget @ self.X_BrickTargetPreG,
                                     self.finger_closed,
                                     'pre-put')
        self.trajectory.append_point(2,
                                     X_WBrickTarget @ self.X_BrickTargetG,
                                     self.finger_closed,
                                     'put')
        self.trajectory.append_point(2.5,
                                     X_WBrickTarget @ self.X_BrickTargetG,
                                     self.finger_opened,
                                     'put and open')
        self.trajectory.append_point(4.5,
                                     X_WBrickTarget @ self.X_BrickTargetPreG,
                                     self.finger_closed,
                                     'pre-put')

    def gen_return_to_source_traj(self, X_WBrickTarget):
        self.trajectory.append_point(0.0001,
                                     X_WBrickTarget @ self.X_BrickTargetPreG,
                                     self.finger_opened,
                                     'go_back_begin')
        self.trajectory.append_point(4,
                                     self.X_WBrickSource @ self.X_BrickSourcePreG,
                                     self.finger_opened,
                                     'go_back_end/calibration',
                                     True)
        
    def get_trajectories(self):
        return self.trajectory


#
# Helper function
#
def put_bricks_in_warehouse(painter, context, wh_location, wh_size, brick_cnt):
    # Put all bricks in the warehouse and lock them there
    # Grid cell spec: each warehouse cell is slightly bigger than the bricks
    b_wh_cell = np.array([0.12, 0.18, 0.12])
    brick_cnt = brick_cnt - 1
    for z in range (0, wh_size[2]):
        for i in range (0, wh_size[1]):
            for j in range (0, wh_size[0]):
                X_WBrickWareHouse = RigidTransform(wh_location + b_wh_cell * np.array([i, j, z]))
                painter.move_brick(context, X_WBrickWareHouse, brick_cnt)
                painter.lock_brick(context, brick_cnt)
                if brick_cnt == 0:
                    return
                else:
                    brick_cnt = brick_cnt - 1


In [42]:
# Instantiate the robot (TODO: remove it from here)
painter = IIWA_Painter()

In [46]:
# Build trajectories
trj_builder = TrajectoryBuilder(painter, X_WBrickSource)
trj_builder.gen_initial_traj()

targets = [np.array([-0.3, 0.3, 0.05])]
z = 0
y = 0
for i in range (1, kNumberOfBricks):
    #
    if i % 3 == 0:
        y = 0
        z = z + 0.1
    else:
        y = y + 0.170
    #
    targets.append(np.array([-0.3, 0.3 + y, 0.05 + z]))

for i in range(0, kNumberOfBricks):
    # Compute destination and orientation
    #if i%2:
    #    X_WBrickTarget = RigidTransform(np.array([-0.3, -0.3, 0.35]))
    #else:
    #    X_WBrickTarget = RigidTransform(np.array([-0.3, 0.3, 0.35]))
    X_WBrickTarget = RigidTransform(targets[i])

    # Generate trajectory
    trj_builder.gen_grab_brick_traj()
    trj_builder.gen_move_to_place_traj(X_WBrickTarget)
    trj_builder.gen_place_brick_traj(X_WBrickTarget)
    trj_builder.gen_return_to_source_traj(X_WBrickTarget)

# Speed-up a bit
trj_builder.get_trajectories().slow_down(1)

# Form iiwa trajectories
traj, finger_traj = trj_builder.get_trajectories().form_iiwa_traj()

In [47]:
# Create robot
painter = IIWA_Painter([traj, finger_traj])

# Create default context
context = painter.CreateDefaultContext()

# Put all bricks in the warehouse
put_bricks_in_warehouse(painter, context, kWhLocation, kWhSize, kNumberOfBricks)

In [ ]:
#
# Move
#
# Move to calibration point and calibrate
sim, ctx = painter.paint(None, context, sim_duration=trj_builder.get_trajectories().get_breakpoints()[0])
q0 = painter.get_q0(ctx)
print("calibration q0= ", q0)

# Continue for all bricks
for i in range(0, kNumberOfBricks):
    # Reset integrator with calibrated value
    painter.reset_integrator(ctx, q0)

    # Lock prev brick at target (if any)
    if (i > 1):
        painter.lock_brick(context, i - 2)

    # Teleport the next new brick from the warehouse and unlock it
    painter.move_brick(context, X_WBrickSource, i)
    painter.unlock_brick(context, i)

    # Manipulate the brick to target
    sim, ctx = painter.paint(sim, ctx, sim_duration=trj_builder.get_trajectories().get_breakpoints()[i+1])

calibration q0=  [ 7.89016517e-01  8.76352817e-01  2.20568506e-03 -7.62719441e-01
 -2.89596482e-03  1.50446814e+00  2.35959301e+00]
